In [18]:
from selenium import webdriver
import time
import os
import shutil
import glob
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import openpyxl
import re
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium.webdriver.common.keys import Keys
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import sys
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

In [19]:
class Scrapper:
    def __init__(self):
        self.download_dir = "./master_export/"
    
    def create_download_folder(self):
        if os.path.isdir(self.download_dir):
            shutil.rmtree(self.download_dir)
            os.mkdir(self.download_dir)
            
    def create_driver(self):
        chrome_options = webdriver.ChromeOptions()
        preferences = {"download.default_directory": self.download_dir,
                       "directory_upgrade": True,
                       "safebrowsing.enabled": True }
        chrome_options.add_experimental_option("prefs", preferences)
        chrome_options.add_argument("--start-maximized")
        # chrome_options.add_argument("--headless");
        self.driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)
    
    def find_element_css(self, css, content=None,all_=False):
        if content is None:
            content = self.driver
            
        attempts = 5
        while attempts:
            try:
                if not all_:
                    element = content.find_element_by_css_selector(css)
                else:
                    element = content.find_elements_by_css_selector(css)
            except Exception as e:
                print(e)
                attempts = attempts - 1
                time.sleep(3)
            else:
                return element
            
    def find_element_tag(self,tag, content=None, all_=False):
        if content is None:
            content = self.driver
            
        attempts = 5
        while attempts:
            try:
                if not all_:
                    element = content.find_element_by_tag_name(tag)
                else:
                    element = content.find_elements_by_tag_name(tag)
            except Exception as e:
                print(e)
                attempts = attempts - 1
                time.sleep(3)
            else:
                return element
            
    def find_element_xpath(self,xpath, content=None, all_=False):
        if content is None:
            content = self.driver
            
        attempts = 5
        while attempts:
            try:
                if not all_:
                    element = content.find_element_by_xpath(xpath)
                else:
                    element = content.find_elements_by_xpath(xpath)
            except Exception as e:
                print(e)
                attempts = attempts - 1
                time.sleep(3)
            else:
                return element
            
            
    def shortForm(self, name):
        first_name = name.split('-')[0]
        words = first_name.split(' ')
        result = ''
        for word in words:
            if len(word)>0:
                result = result+word[0]
        return result.lower()  
        
    def login(self):
        self.driver.get('https://app.apptweak.com')
        email_input = self.find_element_css('#user_email')
        pass_input  = self.find_element_css('#user_password')
        email_input.send_keys('email')
        pass_input.send_keys('password\n')
    
    def find_all_apps(self):
        self.all_apps_link = []
        apps_table = self.find_element_css('#all_apps > table > tbody')
        table_rows = self.find_element_tag('tr',content=apps_table,all_=True)
        for row in table_rows:
            self.all_apps_link.append(self.find_element_tag('td',content=row).find_element_by_tag_name('a').get_attribute('href'))
    
    def export_and_download(self):
        #export the file
        self.find_element_css('#export_daterange').click() 
        self.find_element_css('body > div.daterangepicker.ltr.show-ranges.opensleft > div.ranges > ul > li:nth-child(7)').click()
       
        print('Waiting for export to complete...')
        while True:
            export_count = self.find_element_css('#dl-count')
            if export_count.text != '':
                if int(export_count.text) > 0:
                    break;
            time.sleep(5)
        #download the file   
        print('Export completed, Starting to download file...')
        self.driver.get('https://app.apptweak.com/dashboard/dashboard/exports')  
        self.find_element_css('#pdf_exports > table > tbody > tr:nth-child(1) > td:nth-child(3) > a').click()        
        while True:
            try:
                self.filename = glob.glob(self.download_dir+'*.xlsx',recursive=True)[0]
                print(self.filename)
                print('Download completed')
                break
                
            except IndexError as e:
                print('waiting for download to complete');
                time.sleep(5)

            except Exception as e:
                print(e)
                break
    
    def remove_legacy(self):
        wb_local = openpyxl.load_workbook(self.filename)
        reg_legacy  = re.compile('.*\(legacy\)')
        for sheet in wb_local:
            if re.match(reg_legacy,sheet.title):
                print('deleting sheet {}'.format(sheet.title))
                wb_local.remove(sheet)
            else:
                sheet.title = self.current_prefix+'_'+sheet.title
                
        if not os.path.isdir(self.download_dir+self.current_app):
            os.mkdir(self.download_dir+self.current_app)

        wb_local.save(self.download_dir+self.current_app+'/'+self.current_prefix+'_competitor_analysis.xlsx')
        os.remove(self.filename)
    
    def authincate_google(self):
        attempts = 5
        while attempts:
            try:
                gauth = GoogleAuth()
                gauth.LoadCredentialsFile("mycreds.txt")
                if gauth.credentials is None:
                    gauth.LocalWebserverAuth()
                elif gauth.access_token_expired:
                    gauth.Refresh()
                else:
                    gauth.Authorize()

                gauth.SaveCredentialsFile("mycreds.txt")
                self.drive = GoogleDrive(gauth)
                scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
                creds = ServiceAccountCredentials.from_json_keyfile_name('./testProject-9d72869365a0.json', scope)
                self.gc = gspread.authorize(creds)
                break
            except Exception as e:
                print(e)
    
    def upload_file(self):
        self.file_drive = self.drive.CreateFile({'title':self.current_prefix+'_competitor_analysis'})
        self.file_drive.SetContentFile(self.download_dir+self.current_app+'/'+self.current_prefix+'_competitor_analysis.xlsx') 
        self.file_drive.Upload({'convert': True})
        attempts = 5
        while attempts:
            try:
                self.file_drive.InsertPermission({
                    'type': 'user',
                    'value':'rajivsah@testproject-226604.iam.gserviceaccount.com',
                    'role': 'writer'
                })
            except Exception as e:
                attempts = attempts-1
                print('sharing permission not granted resharing in 10 sec')
                time.sleep(10)
            else:
                break
    
    def keyword_density_analysis(self):
        self.driver.get(self.current_link)
        self.find_element_css('#side-nav > li:nth-child(6) > a').click()
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.pl0 > div > button > span.filter-option.pull-left').click()
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.pl0 > div > div > ul > li:nth-child(4) > a > span.text').click()
        
        #extract column names from first row
        table = self.find_element_css('#keywords-weight-analysis-description > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        thead = self.find_element_tag('thead', content=table)
        first_row = self.find_element_tag('tr',content=thead)
        imgs = self.find_element_tag('img',content=first_row,all_=True)
        self.column_name = ['keywords',' ']
        for img in imgs:
            if img.get_attribute('data-original-title'):
                self.column_name.append(img.get_attribute('data-original-title'))
            elif img.get_attribute('title'):
                self.column_name.append(img.get_attribute('title'))
            else:
                self.column_name.append(' ')
        
        print(self.column_name)
        
        #1-word analysis
        #html to pandas table and replacing Nan with 0
        for i, df1 in enumerate(pd.read_html(table_html)):
            df1.columns = self.column_name
            df1.fillna('0%',inplace=True)  

        #2-word analysis
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(3) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-description > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df2 in enumerate(pd.read_html(table_html)):
            df2.columns = self.column_name
            df2.fillna('0%',inplace=True)    

        #3-word analysis
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(4) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-description > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df3 in enumerate(pd.read_html(table_html)):
            df3.columns = self.column_name
            df3.fillna('0%',inplace=True) 
            
        
        attempts = 5
        while attempts:
            try:
                self.wb = self.gc.open(self.current_prefix+'_competitor_analysis')
            except gspread.exceptions.SpreadsheetNotFound as e:
                self.file_drive.InsertPermission({
                    'type': 'user',
                    'value':'rajivsah@testproject-226604.iam.gserviceaccount.com',
                    'role': 'writer'
                })
                attempts = attempts-1
                if attempts == 0:
                    print("failed to access permission",e)
                    sys.exit()
                print(e)
                print('Waiting to give Permission')
                time.sleep(5)
            else:
                break

        #pandas to google spreadsheet
        ws1 = self.wb.add_worksheet(title=self.current_prefix+'_desc_kw_density_1',rows=df1.shape[0]+1,cols=df1.shape[1])
        cell_list = ws1.range(1,1,df1.shape[0]+1,df1.shape[1])
        index = 0
        for r in dataframe_to_rows(df1, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1

        ws1.update_cells(cell_list)

        ws2 = self.wb.add_worksheet(title = self.current_prefix+'_desc_kw_density_2',rows=df2.shape[0]+1,cols=df2.shape[1] )
        cell_list = ws1.range(1,1,df2.shape[0]+1,df2.shape[1])
        index = 0
        for r in dataframe_to_rows(df2, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws2.update_cells(cell_list)



        ws3 = self.wb.add_worksheet(title = self.current_prefix+'_desc_kw_density_3' ,rows=df3.shape[0]+1,cols=df3.shape[1])
        cell_list = ws1.range(1,1,df3.shape[0]+1,df3.shape[1])
        index = 0
        for r in dataframe_to_rows(df3,  header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws3.update_cells(cell_list)
        
    
    def app_name_analysis(self):
        print('statting App Name Analysis')
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.mt10 > ul > li:nth-child(2) > a').click()
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(2) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-name > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df1 in enumerate(pd.read_html(table_html)):
            df1.columns = self.column_name
            df1.fillna('0',inplace=True)
            df1.replace(to_replace='-',value=0,inplace=True)


        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(3) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-name > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df2 in enumerate(pd.read_html(table_html)):
            df2.columns = self.column_name
            df2.fillna('0',inplace=True) 
            df2.replace(to_replace='-',value=0,inplace=True)


        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(4) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-name > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df3 in enumerate(pd.read_html(table_html)):
            df3.columns = self.column_name
            df3.fillna('0',inplace=True) 
            df3.replace(to_replace='-',value=0,inplace=True)


        ws1 = self.wb.add_worksheet(title = self.current_prefix+'_app_kw_density_1',rows=df1.shape[0]+1,cols=df1.shape[1])
        cell_list = ws1.range(1,1,df1.shape[0]+1,df1.shape[1])
        index = 0
        for r in dataframe_to_rows(df1, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws1.update_cells(cell_list)

        ws2 = self.wb.add_worksheet(title = self.current_prefix+'_app_kw_density_2' ,rows=df2.shape[0]+1,cols=df2.shape[1])
        cell_list = ws1.range(1,1,df2.shape[0]+1,df2.shape[1])
        index = 0
        for r in dataframe_to_rows(df2, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws2.update_cells(cell_list)

        ws3 = self.wb.add_worksheet(title = self.current_prefix+'_app_kw_density_3' ,rows=df3.shape[0]+1,cols=df3.shape[1])
        cell_list = ws1.range(1,1,df3.shape[0]+1,df3.shape[1])
        index = 0
        for r in dataframe_to_rows(df3,  header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws3.update_cells(cell_list)
    
    def subtitle_analysis(self):
        print('Starting Subtitle Analysis')
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.mt10 > ul > li:nth-child(3) > a').click()
        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(2) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-subtitle > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df1 in enumerate(pd.read_html(table_html)):
            df1.columns = self.column_name
            df1.fillna('0',inplace=True)
            df1.replace(to_replace='-',value=0,inplace=True)


        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(3) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-subtitle > div > div > table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df2 in enumerate(pd.read_html(table_html)):
            df2.columns = self.column_name
            df2.fillna('0',inplace=True) 
            df2.replace(to_replace='-',value=0,inplace=True)


        self.find_element_css('#keywords-weight-analysis > div.aptwk_box.aptwk_box--plain.aptwk_box--header.clearfix > div.col-xs-6.text-right.pr0 > form > label:nth-child(4) > input').click()
        table = self.find_element_css('#keywords-weight-analysis-subtitle > div > div> table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df3 in enumerate(pd.read_html(table_html)):
            df3.columns = self.column_name
            df3.fillna('0',inplace=True)  
            df3.replace(to_replace='-',value=0,inplace=True)


        ws1 = self.wb.add_worksheet(title=self.current_prefix+'_subtitle_kw_density_1',rows=df1.shape[0]+1,cols=df1.shape[1])
        cell_list = ws1.range(1,1,df1.shape[0]+1,df1.shape[1])
        index = 0
        for r in dataframe_to_rows(df1, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws1.update_cells(cell_list)

        ws2 = self.wb.add_worksheet(title = self.current_prefix+'_subtitle_kw_density_2' ,rows=df2.shape[0]+1,cols=df2.shape[1])
        cell_list = ws1.range(1,1,df2.shape[0]+1,df2.shape[1])
        index = 0
        for r in dataframe_to_rows(df2, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws2.update_cells(cell_list)

        ws3 = self.wb.add_worksheet(title = self.current_prefix+'_subtitle_kw_density_3' ,rows=df3.shape[0]+1,cols=df3.shape[1])
        cell_list = ws1.range(1,1,df3.shape[0]+1,df3.shape[1])
        index = 0
        for r in dataframe_to_rows(df3,  header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws3.update_cells(cell_list)
        
    
    def promotional_text_analysis(self):
        print('Starting Promotional text analysis')
        self.find_element_css('#side-nav > li:nth-child(4) > a').click()
        table = self.find_element_css('#promotional_text-table')
        followed_apps = self.column_name[2:len(self.column_name)]
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        for i, df1 in enumerate(pd.read_html(table_html)):
            df1.columns = ['App names', 'Promotional text','characters']
            df1.fillna('0',inplace=True) 
            df1.iloc[:,0] = followed_apps
            df1.replace(to_replace='-',value=0,inplace=True)
            counts = []
            for count in df1.iloc[:,2].values:
                counts.append(count.split(' ')[0])
            df1.iloc[:,2] = counts

        ws1 = self.wb.add_worksheet(title=self.current_prefix+'_promo_text',rows = df1.shape[0]+1,cols=df1.shape[1])
        cell_list = ws1.range(1,1,df1.shape[0]+1,df1.shape[1])
        index = 0
        for r in dataframe_to_rows(df1, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws1.update_cells(cell_list)
    
    def subtitle_text_analysis(self):
        print('Starting Subtitle text')
        self.find_element_css('#side-nav > li:nth-child(3) > a').click()
        table = self.find_element_css('#subtitle-table')
        table_html = '<table>'+ table.get_attribute('innerHTML')+'</table>'
        followed_apps = self.column_name[2:len(self.column_name)]
        for i, df1 in enumerate(pd.read_html(table_html)):
            df1.columns = [ 'App names', 'Subtitle','characters']
            df1.replace(to_replace='-',value=0,inplace=True)
            df1.fillna('0',inplace=True) 
            df1.iloc[:,0] = followed_apps
            counts = []
            for count in df1.iloc[:,2].values:
                counts.append(count.split(' ')[0])
            df1.iloc[:,2] = counts

        ws1 = self.wb.add_worksheet(title=self.current_prefix+'_subtitle_text',rows=df1.shape[0]+1,cols=df1.shape[1])
        cell_list = ws1.range(1,1,df1.shape[0]+1,df1.shape[1])
        index = 0
        for r in dataframe_to_rows(df1, header=True, index=False):
            for c in r:
                cell_list[index].value = c
                index = index+1
        ws1.update_cells(cell_list)
        
    def description_analysis(self):
        print('Starting Description Analysis')
        wb_temp = openpyxl.load_workbook(self.download_dir+self.current_app+'/'+self.current_prefix+'_competitor_analysis.xlsx')
        detail_sheet = wb_temp[self.current_prefix+'_Description']
        ws = self.wb.add_worksheet(self.current_prefix+'_desc_word_count',rows=detail_sheet.max_row,cols=2)
        cell_list = ws.range(detail_sheet.dimensions)
        index = 0
        for row1,row2 in zip(detail_sheet['A'],detail_sheet['D']):
            for row in [row1,row2]:
                cell_list[index].value = row.value
                index = index+1

        ws.update_cells(cell_list)
        ws.update_acell('B1','Count') 
        
    def rating_review_analysis(self):
        print('Starting Rating and Reviews Count')
        self.find_element_css('#reviews').click()
        ws = self.wb.add_worksheet(title=self.current_prefix+'_rating_rewiew_count',rows=0,cols=0)
        ws.append_row(['App Name', 'Average Rating', 'Review Count'])
        avg_rating = self.find_element_css('#summary_section > div:nth-child(1) > div > div:nth-child(1) > div > h2').text
        rating_count = self.find_element_css('#summary_section > div:nth-child(1) > div > div:nth-child(2) > div > h2').text
        ws.append_row([self.current_app,avg_rating,rating_count])

        compet_apps_link = []
        for app in self.find_element_css('#competitors-list > ul').find_elements_by_tag_name('li'):
            compet_apps_link.append(app.find_element_by_tag_name('a').get_attribute('href'))


        for compt_link in compet_apps_link:
            self.driver.get(compt_link)
            self.find_element_css('#reviews').click()
            self.current_app = self.find_element_css('#maincontent > div:nth-child(2) > div > div > div.profile-header.row > div > h2').text
            avg_rating = self.find_element_css('#summary_section > div:nth-child(1) > div > div:nth-child(1) > div > h2').text
            rating_count = self.find_element_css('#summary_section > div:nth-child(1) > div > div:nth-child(2) > div > h2').text
            ws.append_row([self.current_app,avg_rating,rating_count])

    def comprtitor_analysis(self):
        print('Starting Competitor Ranking')
        self.driver.get(self.current_link)
        self.find_element_css('#keywords').click()
        self.find_element_css('#kw_competition_tab_link').click()
        time.sleep(10)
        self.find_element_xpath('//*[@id="competitors_ranking"]/div[1]/div/div[3]/button[2]').click()
        self.find_element_xpath('/html/body/div[4]/div[1]/ul/li[7]').click()
        print('Waiting for export to complete...')
        while True:
            export_count = self.find_element_css('#dl-count')
            if export_count.text != '':
                if int(export_count.text) > 0:
                    break;
            time.sleep(5)
        print('Export completed, Starting to download file...')
        self.driver.get('https://app.apptweak.com/dashboard/dashboard/exports')
        self.find_element_css('#pdf_exports > table > tbody > tr:nth-child(1) > td:nth-child(3) > a').click()


        while True:
            try:
                filename = glob.glob(self.download_dir+'*.xlsx',recursive=True)[0]
            except IndexError as e:
                print('waiting for download to complete');
                time.sleep(5)
            except Exception as e:
                print(e)
                break
            else:
                print(filename)
                self.wb_temp = openpyxl.load_workbook(filename)
                ws_temp = self.wb_temp['Keywords']    
                ws = self.wb.add_worksheet(title=self.current_prefix+'_comp_ranking',rows=ws_temp.max_row,cols=ws_temp.max_column)
                cell_list = ws.range(ws_temp.dimensions)
                index = 0
                for row in ws_temp:
                    for cell in row:
                        cell_list[index].value = cell.value
                        index = index+1

                ws.update_cells(cell_list)
                os.remove(filename)
                break

        
        
    
        
    def scrape_a_link(self,link):
        self.current_link = link 
        self.driver.get(self.current_link)
        self.current_app = self.find_element_css('#maincontent > div:nth-child(2) > div > div > div.profile-header.row > div > h2').text
        self.current_prefix = self.shortForm(self.current_app)
        self.export_and_download()
        self.remove_legacy()
        self.authincate_google()
        self.upload_file()
        self.keyword_density_analysis()
        self.app_name_analysis()
        self.subtitle_analysis()
        self.promotional_text_analysis()
        self.subtitle_text_analysis()
        self.rating_review_analysis()
        self.comprtitor_analysis()
        
    
    def scrape(self):
        self.create_download_folder()
        self.create_driver()
        self.login()
        self.find_all_apps()
        for link in self.all_apps_link:
            self.scrape_a_link(link)
        
        


In [20]:
temp = Scrapper()
temp.scrape()

Waiting for export to complete...
Export completed, Starting to download file...
waiting for download to complete
./master_export/2018-12-26-12-17-39-itunes-germany-de-wettdeals-sportwetten-wetten-export.xlsx
Download completed
deleting sheet All Category Ranking (legacy)
deleting sheet Category Ranking (legacy)
deleting sheet Keywords  (legacy)
['keywords', ' ', 'Wettdeals Sportwetten & Wetten', 'Tipico Sports']
statting App Name Analysis
Starting Subtitle Analysis
Starting Promotional text analysis
Starting Subtitle text
Starting Rating and Reviews Count
Starting Competitor Ranking
Waiting for export to complete...
Export completed, Starting to download file...
waiting for download to complete
waiting for download to complete
./master_export/2018-12-26-12-20-02-itunes-germany-de-wettdeals-sportwetten-wetten-keyword-export.xlsx
Waiting for export to complete...
Export completed, Starting to download file...
waiting for download to complete
./master_export/2018-12-26-12-21-25-itunes-un